Charalampos Kaidos & Ioannis Papantonis

In the same directory as this notebook, create a folder named "mnist" and put inside the mnist data that where shipped with the project anouncement. In a separate folder called "cifar-10-batches-py" put the cifar10 pickled dataset as provided for python in the CIFAR website.

The functions bellow load MNIST data set from the files provided in the excercise. Also there are functions to save the data in pickled format and load them from pickled format which is much faster than loading them from the texr files.

In [1]:
from os import listdir
from os.path import isfile, join

import numpy
import pickle

def load_mnist(path):
    print('Reading MNIST dataset from storage')

    trainfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f)) and 'train' in f]
    train_data, train_target = load_mnist_from_files(trainfiles)

    testfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f)) and 'test' in f]
    test_data, test_target = load_mnist_from_files(testfiles)

    print('Finished reading MNIST')
    return train_data, train_target, test_data, test_target

def load_mnist_from_files(files):
    data = None
    target = None

    for file in files:
        print('Reading file {}'.format(file))
        temp_data = numpy.loadtxt(file)
        if (numpy.any(data) == None):
            data = temp_data
        else:
            data = numpy.append(data, temp_data, axis=0)

        rows, columns = temp_data.shape
        temp_target = numpy.zeros((rows, 10))
        temp_target[:, int(file[-5])] = numpy.ones(rows)
        if (numpy.any(target) == None):
            target = temp_target
        else:
            target = numpy.append(target, temp_target, axis=0)

        print('Read {} rows, {} columns from {}'.format(rows, columns, file))

    return data, target

def pickle_mnist(traindata, traintarget, testdata, testtarget):
    f=open('./mnist/trdata', mode='wb+')
    pickle.dump(traindata, f)
    f.close()
    f = open('./mnist/trtarget', mode='wb+')
    pickle.dump(traintarget, f)
    f.close()
    f = open('./mnist/tedata', mode='wb+')
    pickle.dump(testdata, f)
    f.close
    f = open('./mnist/tetarget', mode='wb+')
    pickle.dump(testtarget, f)
    f.close()

def unpickle_mnist():
    fo = open('./mnist/trdata', 'rb')
    traindata = pickle.load(fo)
    fo.close()
    fo = open('./mnist/trtarget', 'rb')
    traintarget = pickle.load(fo)
    fo.close()
    fo = open('./mnist/tedata', 'rb')
    testdata = pickle.load(fo)
    fo.close()
    fo = open('./mnist/tetarget', 'rb')
    testtarget = pickle.load(fo)
    fo.close()
    return traindata, traintarget, testdata, testtarget

The functions bellow load the CIFAR-10 data set from the pickled format provided on the CIFAR website.

In [2]:
from os import listdir
from os.path import isfile, join

import pickle
import numpy

def load_cifar(path):
    print('Reading CIFAR-10 dataset from storage')

    dicts = []

    trainfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f)) and 'data_batch' in f]
    for f in trainfiles:
        dicts.append(unpickle(f))

    data = numpy.vstack(tuple([x['data'] for x in dicts]))
    ones = numpy.ones((data.shape[0], 1))
    data = numpy.hstack((ones,data))

    temp_target = numpy.hstack(tuple([x['labels'] for x in dicts]))
    target = numpy.zeros((temp_target.shape[0], 10))
    for r in range(temp_target.shape[0]):
        target[r, temp_target[r]] = 1

    test_data = unpickle(join(path, 'test_batch'))
    test_target = numpy.zeros((len(test_data['labels']), 10))
    for r in range(test_target.shape[0]):
        test_target[r, test_data['labels'][r]] = 1
    test_data = test_data['data']
    ones = numpy.ones((test_data.shape[0], 1))
    test_data = numpy.hstack((ones, test_data))

    return data, target, test_data, test_target

def unpickle(file):
    fo = open(file, 'rb')
    dictionary = pickle.load(fo, encoding='latin1')
    fo.close()
    return dictionary

The functions bellow implement the gradient descent algorithm.

In [3]:
import numpy
import pickle
import math
import time
from collections import deque


def likelihood_function(W, T, Y, lam):
    '''Calculate the cost for the given W'''
    return numpy.sum(T * numpy.log(Y + numpy.finfo(float).eps)) - lam / 2 * numpy.sum(numpy.linalg.norm(W, axis=0) ** 2)


def softmax(W, X):
    ''' Calculate softmax function for A=XW'''
    Y = numpy.dot(X, W.T)
    Y = Y - numpy.amax(Y, axis=1).reshape(Y.shape[0], 1)
    Y = numpy.exp(Y)
    Y = Y / numpy.sum(Y, axis=1).reshape(Y.shape[0], 1)
    return Y


def gradient_descent(X, T, step, iterations, precision):
    ''' Gradient descent to find optimal weights given a training set X,T
        We use momentum to accelerate the convergence'''

    def init_weights():
        numpy.random.seed(5)
        return deque((numpy.random.randn(T.shape[1], X.shape[1]), None, None))

    def init_momentum():
        return deque((0, None, None))

    def init_cost():
        return deque((-numpy.inf, -numpy.inf, -numpy.inf))

    # Initialize parameters
    W = init_weights()
    momentum = init_momentum()
    cost = init_cost()

    time_elapsed = 0
    for i in range(iterations):
        start = time.time()
        print('Starting iteration {}'.format(i))

        # Calculate the output for the current weights
        Y = softmax(W[0], X)

        # Calculate the cost function for these outputs
        cost.rotate(1)
        cost[0] = likelihood_function(W[0], T, Y, 0.05)

        # If the cost function has decreased increase the learning rate. Otherwise we moved over the "valey", so step
        # back to the previous iteration and cut the learning rate in half. Also if the cost difference between the
        # last 3 iterations is less than the precision we have converged and we should stop
        cost_diff = cost[0] - cost[1]
        if (cost_diff >= 0):
            if (cost_diff > 0 and cost_diff < precision and (cost[1] - cost[2] < precision)):
                break
            else:
                step = step * 1.05
        else:
            step = step * 0.5
            W.rotate(-1)
            cost.rotate(-1)
            momentum.rotate(-1)
            print('Stepping back: {}'.format(cost[0]))
            continue

        # Calculate the gradient
        gradient = numpy.dot(numpy.transpose(T - Y), X) - (0.05 * W[0])

        # Calculate the momentum. Momentum makes changing direction when descenting harder. This makes the algorithm
        # bump less on the "walls" of a "valey" when descenting.
        momentum.rotate(1)
        momentum[0] = 0.5 * momentum[1] + (step * gradient)

        # Adjust the weights according to the momentum calculated
        W.rotate(1)
        W[0] = W[1] + (step * gradient)

        time_elapsed += (time.time() - start)

        print('Gradient norm: {}, Likelihood: {}, step: {}'.format(numpy.linalg.norm(gradient, ord=2), cost[0], step))
        print('Elapsed: {:.3f}s, Remaining: {:.3f}s'.format(time_elapsed,
                                                            (time_elapsed / (i + 1)) * (iterations - i + 1)))

        # If we did something numerically wrong, then NaNs will appear. In this case we should stop
        if (numpy.isnan(numpy.min(W[0]))):
            print('NaNs appear')
            break

    return W[0]

def fuzzy_target(traintarget):
    ''' Transform the indicators from 0 and 1 to values close to 0 and 1
    This seems to work better for some algorithms'''
    for row in range(traintarget.shape[0]):
        for col in range(traintarget.shape[1]):
            if (traintarget[row, col] == 0):
                traintarget[row, col] = 0.0001
            else:
                traintarget[row, col] = 0.9991
    return traintarget

Bellow we load the MNIST data set and execute the gradient descent. In this notebook we executed 10 iterations as example. We have executed 2000 iterations and achieved 91% accuracy.

    Starting iteration 1995
    Gradient norm: 105.9549758565084, Likelihood: -19176.325895635244, step: 2.542919593969382e-05
    Elapsed: 363.595s, Remaining: 1.093s
    Starting iteration 1996
    Gradient norm: 105.80861520577044, Likelihood: -19174.624621548784, step: 2.670065573667851e-05
    Elapsed: 363.789s, Remaining: 0.911s
    Starting iteration 1997
    Gradient norm: 105.79296612757267, Likelihood: -19172.839674034956, step: 2.8035688523512437e-05
    Elapsed: 363.981s, Remaining: 0.729s
    Starting iteration 1998
    Gradient norm: 105.71900664024812, Likelihood: -19170.966864003105, step: 2.943747294968806e-05
    Elapsed: 364.175s, Remaining: 0.547s
    Starting iteration 1999
    Gradient norm: 105.6966246769972, Likelihood: -19169.001909664195, step: 3.090934659717246e-05
    Elapsed: 364.368s, Remaining: 0.364s
    Predicted correct 9135 out of 10000

In [6]:
traindata, traintarget, testdata, testtarget = load_mnist('./mnist')
#traindata, traintarget, testdata, testtarget = unpickle_mnist()

traintarget = fuzzy_target(traintarget)

W = gradient_descent(traindata / 255, traintarget, 0.00001, 10, 0.00001) # 10 Iterations
Y = softmax(W, testdata / 255)
res = numpy.argmax(Y, axis=1) - numpy.argmax(testtarget, axis=1)
zeros = 0
for row in range(res.shape[0]):
    if (res[row] == 0):
        zeros = zeros + 1
print('Predicted correct {} out of 10000'.format(zeros))

Reading MNIST dataset from storage
Reading file ./mnist/train5.txt
Read 5421 rows, 784 columns from ./mnist/train5.txt
Reading file ./mnist/train9.txt
Read 5949 rows, 784 columns from ./mnist/train9.txt
Reading file ./mnist/train7.txt
Read 6265 rows, 784 columns from ./mnist/train7.txt
Reading file ./mnist/train1.txt
Read 6742 rows, 784 columns from ./mnist/train1.txt
Reading file ./mnist/train2.txt
Read 5958 rows, 784 columns from ./mnist/train2.txt
Reading file ./mnist/train8.txt
Read 5851 rows, 784 columns from ./mnist/train8.txt
Reading file ./mnist/train6.txt
Read 5918 rows, 784 columns from ./mnist/train6.txt
Reading file ./mnist/train0.txt
Read 5923 rows, 784 columns from ./mnist/train0.txt
Reading file ./mnist/train3.txt
Read 6131 rows, 784 columns from ./mnist/train3.txt
Reading file ./mnist/train4.txt
Read 5842 rows, 784 columns from ./mnist/train4.txt
Reading file ./mnist/test0.txt
Read 980 rows, 784 columns from ./mnist/test0.txt
Reading file ./mnist/test5.txt
Read 892 rows

Bellow we load the CIFAR10 dataset and execute the gradient descent. In this notebook we executed 10 iterations. We have also executed 100,000 iteration and achieved 34% accuracy:

    Starting iteration 99995
    Gradient norm: 340.2301905652409, Likelihood: -93859.43385195508, step: 9.213183965312024e-07
    Elapsed: 70281.315s, Remaining: 4.217s
    Starting iteration 99996
    Gradient norm: 654.7906132198642, Likelihood: -93859.23399771262, step: 9.673843163577625e-07
    Elapsed: 70282.091s, Remaining: 3.514s
    Starting iteration 99997
    Gradient norm: 1472.3313849536962, Likelihood: -93859.23071787521, step: 1.0157535321756506e-06
    Elapsed: 70282.866s, Remaining: 2.811s
    Starting iteration 99998
    Stepping back: -93859.23071787521
    Starting iteration 99999
    Gradient norm: 1472.3313849536962, Likelihood: -93859.23071787521, step: 5.332706043922166e-07
    Elapsed: 70283.399s, Remaining: 1.406s
    Predicted correct 3391 out of 10000

In [7]:
traindata, traintarget, testdata, testtarget = load_cifar('./cifar-10-batches-py')

traintarget = fuzzy_target(traintarget)

W = gradient_descent(traindata / 255, traintarget, 0.00001, 10, 0.00001) # 10 Iterations
Y = softmax(W, testdata / 255)
res = numpy.argmax(Y, axis=1) - numpy.argmax(testtarget, axis=1)
zeros = 0
for row in range(res.shape[0]):
    if (res[row] == 0):
        zeros = zeros + 1
print('Predicted correct {} out of 10000'.format(zeros))

Reading CIFAR-10 dataset from storage
Starting iteration 0
Gradient norm: 1014474.511582108, Likelihood: -1296218.8497923529, step: 1.0500000000000001e-05
Elapsed: 0.590s, Remaining: 6.491s
Starting iteration 1
Gradient norm: 396788.47955721786, Likelihood: -1044234.4758379774, step: 1.1025000000000002e-05
Elapsed: 1.373s, Remaining: 6.866s
Starting iteration 2
Stepping back: -1044234.4758379774
Starting iteration 3
Gradient norm: 396788.47955721786, Likelihood: -1044234.4758379774, step: 5.7881250000000014e-06
Elapsed: 1.959s, Remaining: 3.918s
Starting iteration 4
Stepping back: -1044234.4758379774
Starting iteration 5
Gradient norm: 396788.47955721786, Likelihood: -1044234.4758379774, step: 3.038765625000001e-06
Elapsed: 2.577s, Remaining: 2.577s
Starting iteration 6
Gradient norm: 333578.87065147684, Likelihood: -1013476.6112241243, step: 3.190703906250001e-06
Elapsed: 3.303s, Remaining: 2.359s
Starting iteration 7
Gradient norm: 249901.01778345762, Likelihood: -971541.7241861853, 

The files gradient_descent.py, mnist.py and cifar10.py include the above code that can be executed on the shell